# GHG Emissions Implementation

## Imports

In [ ]:
import os
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Global Variables

In [ ]:
COLAB = True

In [ ]:
ROOT_DIR_PATH = os.path.abspath('..')

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

  ROOT_DIR_PATH = os.path.abspath('drive/MyDrive/Spatial_Finance_Transport/')

ROOT_VEHICLE_DETECTION_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/vehicle_counts/vehicle_counts_')
EMISSIONS_FACTORS_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/uk_emissions_factors.csv')
VEHICLE_COUNTS_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/vehicle_counts/')

COUNT_SITE_PATHS = ['luton_m1_2557A.csv', 'luton_m1_2557B.csv', 'havering_m25_5790A.csv', 'havering_m25_5790B.csv', 
                           'hounslow_m4_2188A.csv', 'hounslow_m4_2188B.csv',
                           'blackburn_30361033.csv', 'blackburn_30361032.csv']

AADT_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/aadt/')
GHG_EMISSIONS_ROOT_PATH = os.path.join(ROOT_DIR_PATH, 'data/predicted/ghg_emissions/')

In [ ]:
VEHICLE_CATEGORIES = ['Passenger Vehicle',
  'Small Car',
  'Bus',
  'Pickup Truck',
  'Utility Truck',
  'Truck',
  'Cargo Truck',
  'Truck w/Box',
  'Truck Tractor',
  'Trailer',
  'Truck w/Flatbed',
  'Truck w/Liquid',
  'Passenger Car'
]

EMISSIONS_CATEGORY_MAPPING = {
    'Passenger Vehicle': 'Petrol cars',
    'Small Car': 'Petrol cars',
    'Pickup Truck': 'Petrol LGVs',
    'Utility Truck': 'Petrol LGVs',
    'Truck': 'Petrol LGVs',
    'Cargo Truck': 'Rigid HGVs',
    'Truck Tractor': 'Rigid HGVs',
    'Trailer': 'Petrol LGVs',
    'Truck w/Flatbed': 'Rigid HGVs',
    'Truck w/Liquid': 'Rigid HGVs',
    'Passenger Car': 'Petrol cars',
    'Truck w/Box': 'Petrol LGVs',
    'Bus': 'Buses'
}


KM_PER_LITRE_MAPPING = {
    'Passenger Vehicle': 20,
    'Passenger Car': 20,
    'Small Car': 20,
    'Petrol LGVs': 3.4,
    'Rigid HGVs': 3.4,
    'Bus': 4
    
}

In [ ]:
# miles
LUTON_ROAD_LENGTH = 2.6
BLACKBURN_ROAD_LENGTH = 8.0
HOUNSLOW_ROAD_LENGTH = 9.8
HAVERING_ROAD_LENGTH = 11.8
TRAFFORD_ROAD_LENGTH = 6.2

In [ ]:
# km
LUTON_ROAD_LENGTH = 4.18
BLACKBURN_ROAD_LENGTH = 12.87
HOUNSLOW_ROAD_LENGTH = 15.77
HAVERING_ROAD_LENGTH = 19
TRAFFORD_ROAD_LENGTH = 9.98

## Helper Functions

In [ ]:
def save_float_to_csv(float_value, column_name, image_id, file_name):
    """
    Save a float value to a CSV file with the specified column name and file name.
    
    Args:
        float_value (float): The float value to be saved.
        column_name (str): The name of the column in the CSV file.
        file_name (str): The name of the CSV file to be saved.
    """
    # Create a DataFrame with a single row and the specified column name and value
    df = pd.DataFrame({'image_id': image_id, column_name: [float_value]})
    
    # Save the DataFrame to a CSV file
    df.to_csv(file_name, index=False)

In [ ]:
def get_files_in_directory(directory):
    """
    Get a list of all files in a directory.

    Args:
        directory (str): Directory path.

    Returns:
        list: List of files in the directory.
    """
    files = []
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            files.append(file_path)
    return files

In [ ]:
def convert_category_names(dataframes_list, mapping_dict):
    """
    Convert category names in a list of DataFrames using a mapping dictionary.

    Args:
        dataframes_list (list): List of DataFrames with 'category_name' column.
        mapping_dict (dict): Dictionary containing mapping of old category names to new category names.

    Returns:
        list: List of DataFrames with updated category names.
    """
    updated_dataframes = []
    for df in dataframes_list:
        df['Vehicle Type'] = df['category_name'].map(mapping_dict)
        updated_dataframes.append(df)
    return updated_dataframes

In [ ]:
def calculate_ghg_emissions(car_category, emission_factors, road_length):
    """
    Calculates GHG emissions for a given car category, emission factors, and road length.
    
    Args:
        car_category (str): Category of the car (e.g., "Small Car", "Midsize Car", etc.).
        emission_factors (dict): Dictionary containing emission factors for different car categories.
        road_length (float): Length of the road segment in kilometers.
        
    Returns:
        float: Total GHG emissions in kilograms for the given car category and road length.
    """
    # Check if the emission factors dictionary contains the given car category
    if car_category not in emission_factors:
        raise ValueError("Car category not found in emission factors dictionary.")
    
    # Get the emission factors for the given car category
    car_emission_factors = emission_factors[car_category]
    
    # Calculate GHG emissions using the emission factors and road length
    ghg_emissions = car_emission_factors['co2'] * road_length + \
                    car_emission_factors['ch4'] * road_length + \
                    car_emission_factors['n2o'] * road_length
    
    return ghg_emissions

In [ ]:
def add_total_column(dataframes_list, other_dataframe):
    """
    Add a 'Total' column from one DataFrame to each DataFrame in a list of DataFrames based on the 'Vehicle Type' column.

    Args:
        dataframes_list (list): List of DataFrames.
        other_dataframe (DataFrame): DataFrame to extract the 'Total' column from.

    Returns:
        list: List of DataFrames with the 'Total' column added.
    """
    updated_dataframes = []
    for df in dataframes_list:
        if 'Vehicle Type' in df.columns and 'Vehicle Type' in other_dataframe.columns:
            total_column = other_dataframe[['Vehicle Type', 'Total']]
            df = df.merge(total_column, on='Vehicle Type', how='left')
        updated_dataframes.append(df)
    return updated_dataframes


In [ ]:
def create_vehicle_type_counts_df(df):
    """
    Count unique vehicle types in a DataFrame and return a DataFrame with columns as vehicle types and
    a single row with counts as values.
    
    Args:
        df (pandas.DataFrame): DataFrame containing the columns: image_id, x_min, x_max, y_min, y_max,
                               category_name, area, Vehicle Type, and Total.
                               
    Returns:
        pandas.DataFrame: DataFrame with columns as vehicle types and a single row with counts as values.
    """
    # Check if "Vehicle Type" column is present in the DataFrame
    if "Vehicle Type" not in df.columns:
        raise ValueError("Column 'Vehicle Type' not found in the DataFrame.")
    
    # Count unique values in "Vehicle Type" column
    vehicle_type_counts = df["Vehicle Type"].value_counts().to_dict()
    
    # Create a DataFrame from the counts dictionary
    counts_df = pd.DataFrame(vehicle_type_counts, index=[0])
    
    return counts_df

## Load Vehicle Detection Data

In [ ]:
dfs = []

vehicle_count_paths = get_files_in_directory(VEHICLE_COUNTS_ROOT_PATH)

for vehicle_count_path in vehicle_count_paths:
  df = pd.read_csv(vehicle_count_path)

  df.name = df.iloc[0]['image_id']
  print(df.name)
  dfs.append(df)

dfs[1].head()

trafford_m60_9083a
blackburn_30361032
havering_m25_5790b
blackburn_30361033
havering_m25_5790a
hounslow_m4_2188a
hounslow_m4_2188b
trafford_m60_9086b
luton_m1_2557a
luton_m1_2557b


,image_id,x_min,x_max,y_min,y_max,category_name,area
0,blackburn_30361032,0,0,0,0,Small Car,0


## Load Emissions Data

In [ ]:
df_emissions_factors = pd.read_csv(EMISSIONS_FACTORS_PATH)

df_emissions_factors['Total'] = df_emissions_factors.sum(axis=1)

df_emissions_factors

<ipython-input-33-77218a245191>:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_emissions_factors['Total'] = df_emissions_factors.sum(axis=1)


,Vehicle Type,NOx,PM10,PM2.5,CO,VOC,NH3,SO2,Benzene,N2O,Total
0,Petrol cars,0.065000,0.001000,0.001000,0.5830,0.090000,0.013000,0.00000,0.002000,0.001000,0.756000
1,Diesel cars,0.517000,0.008000,0.008000,0.0470,0.004000,0.004000,0.00100,0.000000,0.006000,0.595000
2,Petrol LGVs,0.094000,0.001000,0.001000,1.2340,0.085000,0.017000,0.00100,0.002000,0.002000,1.437000
3,Diesel LGVs,0.808000,0.007000,0.007000,0.0580,0.008000,0.005000,0.00100,0.000000,0.006000,0.900000
4,Rigid HGVs,1.428000,0.020000,0.020000,0.4380,0.039000,0.009000,0.00200,0.000000,0.031000,1.987000
5,Artic HGVs,0.609000,0.011000,0.011000,0.2950,0.027000,0.009000,0.00300,0.000000,0.052000,1.017000
6,Buses,2.602000,0.030000,0.030000,0.7530,0.056000,0.008000,0.00300,0.000000,0.032000,3.514000
7,M/cycle,0.078934,0.007047,0.007047,2.3038,0.280368,0.001973,0.00036,0.012611,0.001822,2.693961


## Convert Vehicle Detection into Compatible Emissions Categories

In [ ]:
dfs = convert_category_names(dfs, EMISSIONS_CATEGORY_MAPPING)

dfs[0].head()

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type
0,trafford_m60_9083a,292.568542,303.204315,477.258476,489.027733,Small Car,125,Petrol cars
1,trafford_m60_9083a,437.270111,447.459076,653.021393,662.627777,Small Car,97,Petrol cars


## AADT by Number of Vehicles

### Load AADT data

In [ ]:
aadt_paths = get_files_in_directory(AADT_ROOT_PATH)

for aadt_path in aadt_paths:
  df_aadt = pd.read_csv(aadt_path, sep = ',', skipinitialspace = True)

  print(df_aadt.head())

  for df in dfs:
    if df.iloc[0]['image_id'] == df_aadt.iloc[0]['image_id']:
      df['aadt'] = df_aadt.iloc[0]['aadt']

dfs[0].head()

             image_id      aadt
0  havering_m25_5790b  60603.17
             image_id      aadt
0  havering_m25_5790a  60527.24
            image_id      aadt
0  hounslow_m4_2188b  61842.91
             image_id      aadt
0  blackburn_30361032  31364.52
             image_id      aadt
0  blackburn_30361033  30435.81
            image_id      aadt
0  hounslow_m4_2188a  55681.47
             image_id      aadt
0  trafford_m60_9083a  60938.07
         image_id      aadt
0  luton_m1_2557b  70214.29
         image_id      aadt
0  luton_m1_2557a  71902.71
             image_id     aadt
0  trafford_m60_9086b  61214.7


,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt
0,trafford_m60_9083a,292.568542,303.204315,477.258476,489.027733,Small Car,125,Petrol cars,60938.07
1,trafford_m60_9083a,437.270111,447.459076,653.021393,662.627777,Small Car,97,Petrol cars,60938.07


## Ratio of AADT to each Vehicle Category

In [ ]:
for df in dfs:

  reciprocal_vehicle_count = 1 / len(df)

  if 'aadt' in df:
    aadt = df.iloc[0]['aadt']

    print("image_id: {}, aadt: {}".format(df.iloc[0]['image_id'], aadt))
    df['aadt_vehicles'] = aadt * reciprocal_vehicle_count

dfs[0].head()

image_id: trafford_m60_9083a, aadt: 60938.07
image_id: blackburn_30361032, aadt: 31364.52
image_id: havering_m25_5790b, aadt: 60603.17
image_id: blackburn_30361033, aadt: 30435.81
image_id: havering_m25_5790a, aadt: 60527.24
image_id: hounslow_m4_2188a, aadt: 55681.47
image_id: hounslow_m4_2188b, aadt: 61842.91
image_id: trafford_m60_9086b, aadt: 61214.7
image_id: luton_m1_2557a, aadt: 71902.71
image_id: luton_m1_2557b, aadt: 70214.29


,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt,aadt_vehicles
0,trafford_m60_9083a,292.568542,303.204315,477.258476,489.027733,Small Car,125,Petrol cars,60938.07,30469.035
1,trafford_m60_9083a,437.270111,447.459076,653.021393,662.627777,Small Car,97,Petrol cars,60938.07,30469.035


In [ ]:
dfs[4]

,image_id,x_min,x_max,y_min,y_max,category_name,area,Vehicle Type,aadt,aadt_vehicles
0,havering_m25_5790a,320.712341,328.918640,2617.645081,2630.568848,Small Car,106,Petrol cars,60527.24,4323.374286
1,havering_m25_5790a,211.705811,219.685989,981.659027,993.882812,Small Car,97,Petrol cars,60527.24,4323.374286
2,havering_m25_5790a,197.888626,204.955078,902.768387,917.467621,Small Car,103,Petrol cars,60527.24,4323.374286
3,havering_m25_5790a,195.034286,202.319916,950.091064,961.887604,Small Car,85,Petrol cars,60527.24,4323.374286
4,havering_m25_5790a,313.852905,323.597473,2646.581238,2662.478668,Small Car,154,Petrol cars,60527.24,4323.374286
5,havering_m25_5790a,679.062073,689.404449,3917.796036,3933.449951,Small Car,161,Petrol cars,60527.24,4323.374286
6,havering_m25_5790a,260.900024,269.985474,1946.164459,1965.173828,Bus,172,Buses,60527.24,4323.374286
7,havering_m25_5790a,255.844727,266.521912,1839.606873,1876.990265,Bus,399,Buses,60527.24,4323.374286
8,havering_m25_5790a,302.838135,314.258881,2527.214630,2561.142761,Bus,387,Buses,60527.24,4323.374286
9,havering_m25_5790a,257.297821,268.603455,2003.715622,2045.192604,Bus,468,Buses,60527.24,4323.374286


## Calculate GHG Emissions 


**TODO: Add mapping for mpg of each vehicle type to calculation for improved accuracy**

In [ ]:
# assumptions for calculating ghg emissions
mpg = 25
km_litre = 22.1
gallons_to_litres = 4.55
kg_to_kt = 1e-6

In [ ]:
aadt_emissions = []

for df in dfs:
  la_name_id = df.iloc[0]['image_id']
  ghg_emissions = 0
  LENGTH = 0

  if 'aadt' in df:

    aadt = df.iloc[0]['aadt']

    if la_name_id.find('blackburn') != -1:
      LENGTH = BLACKBURN_ROAD_LENGTH

    elif la_name_id.find('luton') != -1:
      LENGTH = LUTON_ROAD_LENGTH

    elif la_name_id.find('hounslow') != -1:
      LENGTH = HOUNSLOW_ROAD_LENGTH

    elif la_name_id.find('havering') != -1:
      LENGTH = HAVERING_ROAD_LENGTH

    elif la_name_id.find('trafford') != -1:
      LENGTH = 0.61

    for i in range(len(df)):

      gallons = LENGTH * df.iloc[i]['aadt_vehicles'] / mpg

      litres = gallons * gallons_to_litres

      emissions = litres * df_emissions_factors.loc[df_emissions_factors['Vehicle Type'] == df.iloc[i]['Vehicle Type'], 'Total'].values[0]

      ghg_emissions += np.round(emissions)

    ghg_emissions = ghg_emissions * 1e-3

    print("LA Count Site: {}, AADT Prediction: {}, GHG Emissions Prediction: {}".format(la_name_id, aadt, ghg_emissions))

    save_float_to_csv(ghg_emissions, 'ghg_emissions', image_id=la_name_id, file_name=GHG_EMISSIONS_ROOT_PATH+'ghg_emissions_'+la_name_id+'.csv')
    aadt_emissions.append((la_name_id, ghg_emissions))

LA Count Site: trafford_m60_9083a, AADT Prediction: 60938.07, GHG Emissions Prediction: 5.114
LA Count Site: blackburn_30361032, AADT Prediction: 31364.52, GHG Emissions Prediction: 34.524
LA Count Site: havering_m25_5790b, AADT Prediction: 60603.17, GHG Emissions Prediction: 183.268
LA Count Site: blackburn_30361033, AADT Prediction: 30435.81, GHG Emissions Prediction: 33.502
LA Count Site: havering_m25_5790a, AADT Prediction: 60527.24, GHG Emissions Prediction: 264.56
LA Count Site: hounslow_m4_2188a, AADT Prediction: 55681.47, GHG Emissions Prediction: 75.081
LA Count Site: hounslow_m4_2188b, AADT Prediction: 61842.91, GHG Emissions Prediction: 98.413
LA Count Site: trafford_m60_9086b, AADT Prediction: 61214.7, GHG Emissions Prediction: 7.221
LA Count Site: luton_m1_2557a, AADT Prediction: 71902.71, GHG Emissions Prediction: 25.722
LA Count Site: luton_m1_2557b, AADT Prediction: 70214.29, GHG Emissions Prediction: 47.972


In [ ]:
aadt_emissions = []

for df in dfs:
  la_name_id = df.iloc[0]['image_id']
  ghg_emissions = 0
  LENGTH = 0

  if 'aadt' in df:

    aadt = df.iloc[0]['aadt']

    if la_name_id.find('blackburn') != -1:
      LENGTH = BLACKBURN_ROAD_LENGTH

    elif la_name_id.find('luton') != -1:
      LENGTH = LUTON_ROAD_LENGTH

    elif la_name_id.find('hounslow') != -1:
      LENGTH = HOUNSLOW_ROAD_LENGTH

    elif la_name_id.find('havering') != -1:
      LENGTH = HAVERING_ROAD_LENGTH

    for i in range(len(df)):

      litres = LENGTH * df.iloc[i]['aadt_vehicles'] * 365 / km_litre # litres

      emissions = litres * df_emissions_factors.loc[df_emissions_factors['Vehicle Type'] == df.iloc[i]['Vehicle Type'], 'Total'].values[0] # kg co2

      ghg_emissions += np.round(emissions, 1)

    ghg_emissions = ghg_emissions * kg_to_kt

    print("LA Count Site: {}, AADT Prediction: {}, GHG Emissions Prediction: {}".format(la_name_id, aadt, ghg_emissions))

    save_float_to_csv(ghg_emissions, 'ghg_emissions', image_id=la_name_id, file_name=GHG_EMISSIONS_ROOT_PATH+'ghg_emissions_'+la_name_id+'.csv')
    aadt_emissions.append((la_name_id, ghg_emissions))

LA Count Site: trafford_m60_9083a, AADT Prediction: 60938.07, GHG Emissions Prediction: 0.0
LA Count Site: blackburn_30361032, AADT Prediction: 31364.52, GHG Emissions Prediction: 5.0401049
LA Count Site: havering_m25_5790b, AADT Prediction: 60603.17, GHG Emissions Prediction: 26.777350800000022
LA Count Site: blackburn_30361033, AADT Prediction: 30435.81, GHG Emissions Prediction: 4.8908664
LA Count Site: havering_m25_5790a, AADT Prediction: 60527.24, GHG Emissions Prediction: 38.65720459999999
LA Count Site: hounslow_m4_2188a, AADT Prediction: 55681.47, GHG Emissions Prediction: 10.9638926
LA Count Site: hounslow_m4_2188b, AADT Prediction: 61842.91, GHG Emissions Prediction: 14.370915
LA Count Site: trafford_m60_9086b, AADT Prediction: 61214.7, GHG Emissions Prediction: 0.0
LA Count Site: luton_m1_2557a, AADT Prediction: 71902.71, GHG Emissions Prediction: 3.7527006
LA Count Site: luton_m1_2557b, AADT Prediction: 70214.29, GHG Emissions Prediction: 6.998571599999999
